<a href="https://colab.research.google.com/github/JosenildoJunior/Lh_desafio/blob/main/Desafio_indicium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Desafio**

Você foi contratado por um estúdio de Hollywood chamado PProductions, e agora deve fazer uma análise em cima de um banco de dados cinematográfico para orientar qual tipo de filme deve ser o próximo a ser desenvolvido. Lembre-se que há muito dinheiro envolvido, então a análise deve ser muito detalhada e levar em consideração o máximo de fatores possíveis (a introdução de dados externos é permitida - e encorajada).


# **Entregas**

- 1. Faça uma análise exploratória dos dados (EDA), demonstrando as principais características entre as variáveis e apresentando algumas hipóteses relacionadas. Seja criativo!

- 2. Responda também às seguintes perguntas:

  - Qual filme você recomendaria para uma pessoa que você não conhece?
  - Quais são os principais fatores que estão relacionados com alta expectativa de faturamento de um filme?
  - Quais insights podem ser tirados com a coluna Overview? É possível inferir o gênero do filme a partir dessa coluna?

- 3. Explique como você faria a previsão da nota do imdb a partir dos dados. Quais variáveis e/ou suas transformações você utilizou e por quê? Qual tipo de problema estamos resolvendo (regressão, classificação)? Qual modelo melhor se aproxima dos dados e quais seus prós e contras? Qual medida de performance do modelo foi escolhida e por quê?

- 4.Supondo um filme com as seguintes características:

  {'Series_Title': 'The Shawshank Redemption',
  'Released_Year': '1994',
  'Certificate': 'A',
  'Runtime': '142 min',
  'Genre': 'Drama',
  'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
  'Meta_score': 80.0,
  'Director': 'Frank Darabont',
  'Star1': 'Tim Robbins',
  'Star2': 'Morgan Freeman',
  'Star3': 'Bob Gunton',
  'Star4': 'William Sadler',
  'No_of_Votes': 2343110,
  'Gross': '28,341,469'}


Qual seria a nota do IMDB?

- 5. Salve o modelo desenvolvido no formato .pkl.
- 6. A entrega deve ser feita através de um repositório de código público que contenha:

  - README explicando como instalar e executar o projeto
  - Arquivo de requisitos com todos os pacotes utilizados e suas versões
  - Relatórios das análises estatísticas e EDA em PDF, Jupyter Notebook ou semelhante conforme passo 1 e 2.
  - Códigos de modelagem utilizados no passo 3 (pode ser entregue no mesmo Jupyter Notebook).
  - Arquivo .pkl conforme passo 5 acima.
  Todos os códigos produzidos devem seguir as boas práticas de codificação.


# **Análise inicial nos dados**

Nessa etapa, vamos estabelecer um primeiro contato com os nossos dados. Para começar, vamos responder algumas perguntas:



- Quantos dados temos? (Número de linhas e colunas)

- Quais são as colunas numéricas?

- Temos duplicatas na nossa base de dados? Se sim, vamos removê-las.

- Existem dados nulos nesta base de dados? Será que eles indicam algo importante? O que devemos fazer com eles?

- Vamos observar o dicionário dos dados.

- Qual é a média, mediana, 25º percentil, 75º percentil, mínimo e máximo de cada uma das colunas numéricas?

Antes de tudo, vamos importar as bibliotecas que serão necessárias para realizarmos nossas análises.

In [8]:
# Manipulação de dados
import pandas as pd

# Algébra linear
import numpy as np

# Visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# Funções estatísticas
import statistics
from scipy import stats

# Importando o skew
from scipy.stats import skew

Agora vamos importar nossos dados.

In [9]:
# Acesso ao drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
# Importando o dataset
df = pd.read_csv('/content/drive/MyDrive/Dados/desafio_indicium_imdb.csv')

# Observando os primeiro registros
df.head()

,Unnamed: 0,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,1,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
1,2,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
2,3,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
3,4,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
4,5,The Lord of the Rings: The Return of the King,2003,U,201 min,"Action, Adventure, Drama",8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,1642758,"377,845,905"


Nesse primeiro contato podemos notar que nossa base de dados possui diversos, tipos de dados desde numericos a categoricos, dito isso vamos observar as dimensões desse nosso data set  

In [21]:
# Dimensão do nosso DF
df.shape

(999, 16)

Temos um total de 999 linhas e 16 colunas, sabendo disso vamos dar uma olhada no nosso dicionario de dados

Agora vamos observar o dicionário dos dados.

- Series_Title – Nome do filme
- Released_Year - Ano de lançamento
- Certificate - Classificação etária
- Runtime – Tempo de duração
- Genre - Gênero
- IMDB_Rating - Nota do IMDB
- Overview - Overview do filme
- Meta_score - Média ponderada de todas as críticas
- Director – Diretor
- Star1 - Ator/atriz #1
- Star2 - Ator/atriz #2
- Star3 - Ator/atriz #3
- Star4 - Ator/atriz #4
- No_of_Votes - Número de votos
- Gross - Faturamento


Agora que temos o dicionario para nos auxiliar vamos observar o tipo de dado presente em cada uma das nossas colunas

In [22]:
# Observando o tipo dos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     999 non-null    int64  
 1   Series_Title   999 non-null    object 
 2   Released_Year  999 non-null    object 
 3   Certificate    898 non-null    object 
 4   Runtime        999 non-null    object 
 5   Genre          999 non-null    object 
 6   IMDB_Rating    999 non-null    float64
 7   Overview       999 non-null    object 
 8   Meta_score     842 non-null    float64
 9   Director       999 non-null    object 
 10  Star1          999 non-null    object 
 11  Star2          999 non-null    object 
 12  Star3          999 non-null    object 
 13  Star4          999 non-null    object 
 14  No_of_Votes    999 non-null    int64  
 15  Gross          830 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 125.0+ KB


Agora podemos notar que a variavel 'Unnamed' é do tipo inteiro essa variavel normalmente aparece em arquivos do tipo csv ela é algo como um indice sem nome, então vamos seguir para as proximas e dividir nossos dados numericos dos nossos dados categoricas

**Dados númericos**

- A variavel 'IMDB_Rating' é uma variavel numerica do tipo float
- A variavel 'Meta_score' também é numerica do tipo float
- A varaivel 'No_of_Votes' é do tipo numerica e int

**Dados categoricos**

Já os dados categoricos são compostos pelas seguintes variaveis:

- 'Series_Title', 'Released_Year', 'Certificate', 'Runtime', 'Genre', 'Overview ', 'Director', 'Star1', 'Star2', 'Star3', 'Star4', 'Gross'.






Aqui podemos notar que alguns dados categoricos deveriam ser do tipo numerico, como é o exemplo da varaivel 'Gross' que é referente ao faturamento ela deveria ser do tipo float, algo semelhante acontece com a variavel 'Released_Year' que deveria ser do tipo numerico ou do tipo date e também podemos observar a variavel 'Runtime' é o tempo do filme em minutos e esta do tipo categorico.

É possivel realizarmos algumas voncerções são necessarias para que possamos trabalhar melhor com esses dados

#### **Observar os tipos dos dados categoricos**

# **Gerando o requirements**

In [7]:
"""

# Gerar o arquivo de requisitos
!pip freeze > requirements.txt

# Verificar o conteúdo do arquivo de requisitos
!cat requirements.txt

# Baixar o arquivo de requisitos
from google.colab import files
files.download('requirements.txt')

"""

"\n\n# Gerar o arquivo de requisitos\n!pip freeze > requirements.txt\n\n# Verificar o conteúdo do arquivo de requisitos\n!cat requirements.txt\n\n# Baixar o arquivo de requisitos\nfrom google.colab import files\nfiles.download('requirements.txt')\n\n"